## Input Text Within Word Limit of 200 - 400 words

In [1]:
input_text = """[ A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as
an introduction and provides additional context for your application. Here’s a breakdown of its various
aspects:
Purpose
The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context
for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that
your resume may not fully capture. It’s also an opportunity to express your enthusiasm for the role and the
company, and to explain why you would be a good fit.
Content
A typical cover letter includes the following sections:
1. Header: Includes your contact information, the date, and the employer’s contact information.
2. Salutation: A greeting to the hiring manager, preferably personalized with their name.
3. Introduction: Briefly introduces who you are and the position you’re applying for.
4. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and
skills that make you suitable for the job. You can also mention how you can contribute to the company.
5. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include
a call to action, like asking for an interview.
6. Signature: A polite closing (“Sincerely,” “Best regards,” etc.) followed by your name.
Significance in the Job Application Process
The cover letter is often the first document that a hiring manager will read, so it sets the tone for your
entire application. It provides you with a chance to stand out among other applicants and to make a
strong first impression. Some employers specifically require a cover letter, and failing to include one could
result in your application being disregarded.
In summary, a cover letter is an essential component of a job application that serves to introduce you,
elaborate on your qualifications, and make a compelling case for why you should be considered for the
position.] """

## Check for Exceeding Word Limit

In [ ]:
# input_text = """[ A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as
# an introduction and provides additional context for your application. Here’s a breakdown of its various
# aspects:
# Purpose
# The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context
# for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that
# your resume may not fully capture. It’s also an opportunity to express your enthusiasm for the role and the
# company, and to explain why you would be a good fit.
# Content
# A typical cover letter includes the following sections:
# 1. Header: Includes your contact information, the date, and the employer’s contact information.
# 2. Salutation: A greeting to the hiring manager, preferably personalized with their name.
# 3. Introduction: Briefly introduces who you are and the position you’re applying for.
# 4. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and
# skills that make you suitable for the job. You can also mention how you can contribute to the company.
# 5. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include
# a call to action, like asking for an interview.
# 6. Signature: A polite closing (“Sincerely,” “Best regards,” etc.) followed by your name.
# Significance in the Job Application Process
# The cover letter is often the first document that a hiring manager will read, so it sets the tone for your
# entire application. It provides you with a chance to stand out among other applicants and to make a
# strong first impression. Some employers specifically require a cover letter, and failing to include one could
# result in your application being disregarded.
# In summary, a cover letter is an essential component of a job application that serves to introduce you,
# elaborate on your qualifications, and make a compelling case for why you should be considered for the
# position.]
# [ A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as
# an introduction and provides additional context for your application. Here’s a breakdown of its various
# aspects:
# Purpose
# The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context
# for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that
# your resume may not fully capture. It’s also an opportunity to express your enthusiasm for the role and the
# company, and to explain why you would be a good fit.
# Content
# A typical cover letter includes the following sections:
# 1. Header: Includes your contact information, the date, and the employer’s contact information.
# 2. Salutation: A greeting to the hiring manager, preferably personalized with their name.
# 3. Introduction: Briefly introduces who you are and the position you’re applying for.
# 4. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and
# skills that make you suitable for the job. You can also mention how you can contribute to the company.
# 5. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include
# a call to action, like asking for an interview.
# 6. Signature: A polite closing (“Sincerely,” “Best regards,” etc.) followed by your name.
# Significance in the Job Application Process
# The cover letter is often the first document that a hiring manager will read, so it sets the tone for your
# entire application. It provides you with a chance to stand out among other applicants and to make a
# strong first impression. Some employers specifically require a cover letter, and failing to include one could
# result in your application being disregarded.
# In summary, a cover letter is an essential component of a job application that serves to introduce you,
# elaborate on your qualifications, and make a compelling case for why you should be considered for the
# position.]
#  """

## Custom Paraphrase Generation Using T5-base

In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import re
import language_tool_python
from time import time
from nltk.translate.bleu_score import sentence_bleu

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CustomParaphraseGenerator:
    def __init__(self, model_name='t5-base'):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

    def paraphrase(self, text, max_length=512, min_length_ratio=0.8, num_return_sequences=1):
        # Check word limit constraint (200-400 words)
        words = re.findall(r'\b\w+\b', text)
        num_words = len(words)
        if num_words < 200 or num_words > 400:
            raise ValueError("Word limit should be between 200 and 400.")

        # Process text in chunks if it exceeds max_length
        if len(text) > max_length:
            chunks = []
            chunk_size = max_length - 20 
            for i in range(0, len(text), chunk_size):
                chunk = text[i:i + chunk_size]
                chunks.append(chunk)

            paraphrases = []
            for chunk in chunks:
                paraphrase_chunk = self._paraphrase_chunk(chunk, max_length, min_length_ratio, num_return_sequences)
                paraphrases.append(paraphrase_chunk)

            return ' '.join(paraphrases)
        else:
            return self._paraphrase_chunk(text, max_length, min_length_ratio, num_return_sequences)

    def _paraphrase_chunk(self, text, max_length, min_length_ratio, num_return_sequences):
        preprocess_text = "paraphrase: " + text + " </s>"
        tokenized_text = self.tokenizer.encode(preprocess_text, return_tensors="pt", max_length=max_length, truncation=True).to(device)

        min_length = int(len(tokenized_text[0]) * min_length_ratio)
        summary_ids = self.model.generate(
            tokenized_text,
            max_length=max_length,
            min_length=min_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=2,
            early_stopping=True,
            repetition_penalty=2.5,
            length_penalty=1.0,
            num_beams=4
        )

        output = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        output = self.correct_english(output)

        return output

    def correct_english(self, text):
        text = re.sub(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s*', lambda x: x.group(0).capitalize(), text)
        text = re.sub(r'\s([.,!?;:"](?:\s|$))', r'\1', text)
        return text

def correct_text(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    corrected_text = tool.correct(text)
    return corrected_text

def calculate_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    score = sentence_bleu(reference, candidate)
    return score

paraphrase_generator = CustomParaphraseGenerator()

try:

    start_time = time()
    cpg_output = paraphrase_generator.paraphrase(input_text)
    cpg_latency = time() - start_time

    cpg_corrected_output = correct_text(cpg_output)

    print("Custom Paraphrase Generator Latency:", cpg_latency)

except ValueError as ve:
    print(f"Warning: {ve}")

import textwrap

wrapped_text = textwrap.fill(cpg_corrected_output.strip(), width=100)

# Print the wrapped text
print("Custom Paraphrase Generator Output (CPG):\n")
print(wrapped_text)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/seraj/miniconda3/envs/pyt/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:290: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Custom Paraphrase Generator Latency: 32.448073863983154
Custom Paraphrase Generator Output (CPG):

A cover letter serves as an introduction to the hiring manager and provides additional context for
your resume. True story: Cover letters can be used as opportune time for you to explain your
qualifications, skills, and experiences if you aren’t sure what you want to say about yourself or
your company - but not necessarily in terms of how you would like to appear on your own resume; they
can also help you with your interview questions; a cover letter should include the following
sections: 1. Header: Includes your contact information, the date, and the employer’s contact info.
2. Salutation: A greeting to the hiring manager, preferably personalized with their name. 3.
Introduction: Briefly introduces who you are and what you’re applying for. 5. Body: This is the core
of your covering letter. 6. Conclusion: Your conclusion should be clear and concise. A cover letter
is often the first docum

In [ ]:
The cover letter is often the first document that a hiring manager will read, so it sets the tone for your
entire application. It provides you with a chance to stand out among other applicants and to make a
strong first impression. Some employers specifically require a cover letter, and failing to include one could
result in your application being disregarded.
In summary, a cover letter is an essential component of a job application that serves to introduce you,
elaborate on your qualifications, and make a compelling case for why you should be considered for the
position.] 


---------------------------

It's the most important
part of an application, and can be one of the last documents to be considered if you want to apply
for the role he/she wants to take on OEM or other similar roles at the same company as you are
applying for u.s. jobs ID: jimmy-krzyzewski@gmail.com 


cover letter is an essential component of a
job application. It serves to introduce you, elaborate on your qualifications, and make compelling
case for why yogi is the right fit for the job he or she is seeking. In short, it sets the tone for
your entire application; however, failing to include one could result in your application being
disregarded by the hiring manager. This article was originally published in The New York Times.
Falcon: u should be considered for the position.

In [ ]:
The cover letter>512 token

## GPT-2 Model for paraphrase generation

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

class GPT2ParaphraseGenerator:
    def __init__(self, model_name='gpt2', device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        self.model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
        self.device = device

    def paraphrase(self, text, max_length=512, num_return_sequences=1):
        input_ids = self.tokenizer.encode(text, return_tensors="pt", max_length=max_length, truncation=True).to(self.device)

        # Generate paraphrase
        output = self.model.generate(input_ids, max_length=max_length, num_return_sequences=num_return_sequences, no_repeat_ngram_size=2, num_beams=4)

        # Decode the generated text
        decoded_output = self.tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        return decoded_output

gpt2_generator = GPT2ParaphraseGenerator()

gpt2_output = gpt2_generator.paraphrase(input_text)

gpt2_corrected_output = correct_text(gpt2_output)

print("\nGPT-2 Based Paraphrase Generator Output:\n", gpt2_corrected_output)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



GPT-2 Based Paraphrase Generator Output:
 [A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as
an introduction and provides additional context for your application. Here’s a breakdown of its various
aspects:
Purpose
The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context
for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that
your resume may not fully capture. It’s also an opportunity to express your enthusiasm for the role and the
company, and to explain why you would be a good fit.
Content
A typical cover letter includes the following sections:
1. Header: Includes your contact information, the date, and the employer’s contact information.
2. Salutation: A greeting to the hiring manager, preferably personalized with their name.
3. Introduction: Briefly introduces who you are and the position you’re applying for.
4. Body: This is the

## Assessment metrics for comparing paraphrases generated by the T5-base and GPT-2 models.

In [9]:
from rouge import Rouge  # Make sure to install 'rouge' package using `pip install rouge`

# Function to calculate BLEU score
def calculate_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    score = sentence_bleu(reference, candidate)
    return score

# Function to calculate ROUGE score
def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)[0]  # Extract the first (and usually only) score
    return scores

# Calculate BLEU and ROUGE scores
reference = input_text.strip()
bleu_cpg = calculate_bleu(reference, cpg_output)
bleu_gpt2 = calculate_bleu(reference, gpt2_output)

rouge_cpg = calculate_rouge(reference, cpg_output)
rouge_gpt2 = calculate_rouge(reference, gpt2_output)

In [7]:
# Calculate BLEU and ROUGE scores
reference = input_text.strip()
bleu_cpg = calculate_bleu(reference, cpg_output)
bleu_gpt2 = calculate_bleu(reference, gpt2_output)

rouge_cpg = calculate_rouge(reference, cpg_output)
rouge_gpt2 = calculate_rouge(reference, gpt2_output)
print("\nBLEU Score:")
print("CPG:", bleu_cpg)
print("GPT-2:", bleu_gpt2)

print("\nROUGE Score:")
print("CPG:", rouge_cpg)
print("GPT-2:", rouge_gpt2)


BLEU Score:
CPG: 0.3376239055260594
GPT-2: 0.9733330366808299

ROUGE Score:
CPG: {'rouge-1': {'r': 0.5777777777777777, 'p': 0.5942857142857143, 'f': 0.5859154879587384}, 'rouge-2': {'r': 0.3888888888888889, 'p': 0.42424242424242425, 'f': 0.4057970964587272}, 'rouge-l': {'r': 0.55, 'p': 0.5657142857142857, 'f': 0.5577464738742315}}
GPT-2: {'rouge-1': {'r': 1.0, 'p': 0.9574468085106383, 'f': 0.9782608645675803}, 'rouge-2': {'r': 1.0, 'p': 0.9696969696969697, 'f': 0.9846153796165681}, 'rouge-l': {'r': 1.0, 'p': 0.9574468085106383, 'f': 0.9782608645675803}}


In [10]:
# Print scores with full name of ROUGE components
print("\nBLEU Score:")
print("CPG (BLEU):", bleu_cpg)
print("GPT-2 (BLEU):", bleu_gpt2)

print("\nROUGE Score:")
print("CPG (ROUGE score):")
print("  Recall (r):", rouge_cpg['rouge-1']['r'])
print("  Precision (p):", rouge_cpg['rouge-1']['p'])
print("  F1 Score (f):", rouge_cpg['rouge-1']['f'])

print("GPT-2 (ROUGE score):")
print("  Recall (r):", rouge_gpt2['rouge-1']['r'])
print("  Precision (p):", rouge_gpt2['rouge-1']['p'])
print("  F1 Score (f):", rouge_gpt2['rouge-1']['f'])


BLEU Score:
CPG (BLEU): 0.3376239055260594
GPT-2 (BLEU): 0.9733330366808299

ROUGE Score:
CPG (ROUGE score):
  Recall (r): 0.5777777777777777
  Precision (p): 0.5942857142857143
  F1 Score (f): 0.5859154879587384
GPT-2 (ROUGE score):
  Recall (r): 1.0
  Precision (p): 0.9574468085106383
  F1 Score (f): 0.9782608645675803
